# 1. [40] Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

You will score documents with respect to user query using language models and also get some experience with topic modelling.

## 1.0. [5] Loading data

We use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads) ([or from github](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/topic-modelling.zip)).

In [1]:
import os
from nltk import word_tokenize


# TODO: read the dataset
path = "../datasets/topic-modelling/"
_, _, filenames = next(os.walk(path))

labels = {
    "testdata_braininjury_10000docs.txt" : "brain_injury",
    "testdata_news_music_2084docs.txt"   : "music",
    "testdata_news_fuel_845docs.txt"     : "fuel",
    "testdata_news_economy_2073docs.txt" : "news_economy"
}

all_data = []
for filename in filenames:
    with open(path + filename, "r") as f:
        sentences = f.read().split("\n")
        all_data.extend([(word_tokenize(sentence.lower()), labels[filename], doc_id) for doc_id, sentence in enumerate(sentences) if sentence])


print("# of documents", len(all_data))
assert len(all_data) == 15002

# of documents 15002


## 1.1. [35] Ranking Using Language Models
Our goal is to rank documents by $P(d|q)$, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: $P(d|q) = \frac{P(q|d)P(d)}{P(q)}$

$P(q)$ is the same for all documents, and so can be ignored. The prior probability of a document $P(d)$ is often treated as uniform across all $d$'s and so it can also be ignored. What does it mean? 

It means that computing $P(q|d)$ for different documents we can compare how relevant are they to the query. How can we estimate $P(q|d)$?

$P(q|d)$ can be estimated as:

![](https://i.imgur.com/BEIMAC1.png)

where $M_d$ is the language model of document $d$, $tf_{t,d}$ is the term frequency of term $t$ in document $d$, and $L_d$ is the number of tokens in document $d$. That is, we just count up how often each word occurred, and divide by the total number of words in the document $d$.

### 1.1.1. [5] Build TDM (or DTM)

The first thing we need to do is to build a term-document matrix for tour dataset.

In [6]:
# TODO: build term-document matrix for the dataset
from collections import Counter, defaultdict

TDM = defaultdict(Counter)
doc_to_label = {}
doc_to_doc = {}
for sentence, label, doc_id in all_data:
    TDM[doc_id].update(Counter(sentence))
    doc_to_label[doc_id] = label
    doc_to_doc[doc_id] = sentence

### 1.1.2. [15] Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *[additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)* - adding a small number (0 to 1) to the observed counts and renormalizing to give a probability distribution.

Another option is called [Jelinek-Mercer smoothing](http://mlwiki.org/index.php/Smoothing_for_Language_Models#Jelinek-Mercer_Smoothing) - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and $M_c$ is a language model built from the entire document collection.

Refer to [*Chapter 12*](https://nlp.stanford.edu/IR-book/html/htmledition/language-models-for-information-retrieval-1.html) for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes TDM or DTM as an input, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [7]:
import numpy as np

def lm_rank_documents(query, tdm, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    
    scores = defaultdict(lambda: 1.0)
    
    TDM_entire = Counter()
    for doc_counter in TDM.values():
        TDM_entire.update(doc_counter)
    
    if smoothing == 'additive':
        d = len(TDM_entire.keys())
        for doc_id, words_freq in TDM.items():
            doc_length = sum(words_freq.values())
            for word in query:
                scores[doc_id]*=(words_freq[word] + param)/(doc_length + param*d)
    
    if smoothing == "jelinek":    
        for doc_id, words_freq in TDM.items():
            doc_length = sum(words_freq.values())
            entire_length = sum(TDM_entire.values())
            pMd = 1
            pMc = 1
            for word in query:
                pMd*=words_freq[word]/doc_length
                pMc*=TDM_entire[word]/entire_length
                
            scores[doc_id] = (1-param)*pMd + param*pMc
    return scores

### 1.1.3. [15] Testing

Check if this type of ranking gives meaningful results. For each query output document `category`, `doc_id`, `score`, and the *beginning* of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [15]:
def process_query(raw_query):
    # TODO: process user query and print search results including document category, id, score, and some part of it
    query_words = word_tokenize(raw_query.lower())
    results = lm_rank_documents(query_words, TDM, smoothing='additive', param=0.001)
    sorted_res = sorted(list(results.items()), key=lambda x: x[1], reverse=True)
    print("user query: " + raw_query + "\n")
    print("search top-5 results:")
    for doc_id, score in sorted_res[:5]:
        print(doc_to_label[doc_id], doc_id, score)
        print(" ".join(doc_to_doc[doc_id][:13]), " ...")
    

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    process_query(q)
    print("\n")

user query: piano concert

search top-5 results:
news_economy 890 8.984497535964978e-06
the budget that president bush will send congress week from monday strays far  ...
news_economy 916 8.969391519224745e-06
president bush vowed rally the nation with wartime state the union address tuesday  ...
news_economy 670 3.049589815902634e-06
amp princess cruises rejected sweetened bid monday from the carnival corp the world  ...
news_economy 1782 2.041838678712098e-06
jayson williams the former new jersey nets basketball star turned himself the new  ...
news_economy 1994 1.8821552941897665e-06
chrysler said monday that planned make convertible version the cruiser which was instant  ...


user query: symptoms of head trauma

search top-5 results:
brain_injury 9355 2.660886418304183e-11
incidental finding chiari malformation with progression symptoms after head trauma case report chiari  ...
brain_injury 6279 2.4186974341385217e-11
head trauma female professional wrestlers the clinical characte

Sample results can look like this (if collapsed, click on 3 dots):

```
user query: piano concert

search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...


user query: symptoms of head trauma

search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed care population background although ...
brain_injury 6987 0.05854539395767944
history reported head trauma sample women substance abuse treatment objectives determine the prevale...
brain_injury 5257 0.05760140208255336
violent head trauma china report cases background the occurrence violent trauma has recently increas...
brain_injury 1536 0.055365767080148634
mild head trauma and chronic headaches returning soldiers objective determine the incidence and type...
brain_injury 8874 0.05379997937839304
maxillofacial trauma major trauma patients background trauma has been identified major public health...


user query: wall street journal

search results:
economy 11294 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
economy 11295 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
music 14641 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
music 14640 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
economy 11297 0.025763725974814314
these feature stories are moving today clients the new york times news service stories are for relea...
```

## 1.2. [+10/100] Bonus - topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [ ]:
# TODO: apply LDA to our dataset and output the resulting categories 


We expect to see something like this (if collapsed, click on 3 dots):

```
Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
```

# 2. [60] Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. (+) Optionally add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 2.0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [ ]:
!pip install pygtrie

### 2.0.1. Check it works and understand the example

In [16]:
import pygtrie
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 2.1. Build a trie upon a dataset

### 2.1.1. [5] Read dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset.

In [22]:
import pandas as pd
aol_data = pd.read_csv("../datasets/aol/user-ct-test-collection-01.txt", encoding='utf-8', sep="\t")

#TODO: Read the dataset, e.g. as pandas dataframe

assert aol_data.shape[0] == 3558411, "Dataset size doesnt' match"

### 2.1.2. [20] Build Trie

We want suggest function to be **non-sensitive to stop words** because we don't want to upset the user if he confuses/omits prepositions, for example. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build Trie based on the dataset, **storing query statistics such as query frequency, urls and ranks in nodes**. Some queries may not have associated urls, others may have multiple ranked urls. Think of the way to store this information.

In [146]:
from tqdm.notebook import tqdm

term_count = Counter()
query_to_urls_ranks = defaultdict(list)

term_number = len(aol_data)

for _, row in tqdm(aol_data.iterrows(), total=term_number):
    query = row.Query
    term_count.update([str(query)])
    query_to_urls_ranks[query].append((row.ClickURL, row.ItemRank))

In [154]:
from nltk import word_tokenize, download
from nltk.corpus import stopwords
import math
download('stopwords')

#TODO: build trie based on dataset
aol_trie = pygtrie.CharTrie()

stop_words = set(stopwords.words("english"))

for query, query_count in tqdm(term_count.items()):
    cleaned_query = " ".join([word for word in query.split(" ") if word not in stop_words])
    cleaned_urls = [pair for pair in query_to_urls_ranks[query] if not math.isnan(pair[1])]
    aol_trie[cleaned_query] = (query_count/term_number, cleaned_urls)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/osmiyg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a list field. But you can do something different.
    bag += [url for url, _ in val[1]]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ (1.4051215556606586e-06, [('http://www.surveyconnect.com', 7.0), ('http://www.custominsight.com', 4.0), ('http://www.askemployees.com', 10.0), ('http://www.lg-employers.gov.uk', 1.0)])
sample questions immigration interview ~ (2.810243111321317e-07, [])
sample questions interview ~ (2.810243111321317e-07, [('http://www.quintcareers.com', 1.0)])
sample questions family interview ~ (8.430729333963952e-07, [('http://www.grandparents-day.com', 2.0), ('http://www.quintcareers.com', 5.0), ('http://jobsearchtech.about.com', 3.0)])
sample questions sociology race ethnicity ~ (2.810243111321317e-07, [])
sample questions biology ~ (5.620486222642634e-07, [('http://www.utexas.edu', 3.0), ('http://www.troy.k12.ny.us', 6.0)])
sample questions us citizenship test ~ (2.810243111321317e-07, [('http://uscis.gov', 1.0)])
sample questionarie teaching evaluation ~ (2.810243111321317e-07, [])
sample questionnaire teaching evaluation ~ (1.4051215556606586e-06, [('http://www.surveyc

## 2.2. [20] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Q: What is the empirical threshold for minimal prefix for suggest?

In [191]:
def complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    cleaned_query = " ".join([word for word in query.split() if word not in stop_words])
    top_k_results = sorted(aol_trie.iteritems(query), key=lambda x: x[1][0], reverse=True)[:top_k]
    all_urls = []
    for _, url in top_k_results:
        all_urls.extend(url[1])
    
    if all_urls:
        best = sorted(all_urls, key=lambda x: x[1])[0][0]
    
    return (top_k_results[0][0], best)
        
inp = "trie"
print("Query:", inp)
res = complete_user_query(inp, aol_trie)

#NB we assume you return suggested query string only

## NO AND HERE
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

Query: trie
Results:
('tried true tattoo', 'http://www.triedntruetattoo.com')


AssertionError: 

## 2.3. [15] Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [193]:
from time import process_time 
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]
start_all = process_time()
for query in inp_queries:
    start = process_time()
    complete_user_query(query, aol_trie)
    print("Query " + query + " : " + str(process_time() - start))

print("Overral time: " + (start_all - process_time))

#TODO: measure avg execution time (in milliseconds) per query and print it out

('information clearing house', 'http://www.informationclearinghouse.info')
Query inf : 0.022538000000167813


KeyError: 'the best '

## 2.5. [+10/100] Bonus task - add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

In [194]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]


def complete_user_query_with_spellchecker(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    def spellss(s):
        ws = s.split()
        cs = [list(spell.candidates(w)) for w in ws]
        return rec(cs)

    spell = SpellChecker()
    cleaned_query = " ".join([word for word in query.split() if word not in stop_words])


for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)

SyntaxError: invalid syntax (<ipython-input-194-05f58dd0c578>, line 15)